# Profitable App Profiles for the App Store and Google Play Markets
This project analyzes a portfolio of free mobile apps which bring revenue through in-app ads. The revenue for any given app is mostly infuenced by the number of users who use our app -- the more users that see and engage with the adds, the better.

The goal of this project is to guide developers so that they can gain better understanding of what type of apps are likely to attract more users - and hence bring more revenue.

## Exploring Data
We have our data for App Store and Play Store in AppleStore.csv and googleplaystore.csv respectively. Here we sample a few rows and try to figure out which attributes/columns would be useful for our analysis

In [64]:
from csv import reader
iphone=list(reader(open('AppleStore.csv')))
android=list(reader(open('googleplaystore.csv')))
iphone_header=iphone[0]
android_header=android[0]

In [15]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

In [16]:
explore_data(iphone,0,2,True)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


Number of rows: 7198
Number of columns: 16


In [17]:
explore_data(android,0,2,True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


Number of rows: 10842
Number of columns: 13


## Deleting Wrong Data

The Google Play data set has a dedicated [discussion section](https://www.kaggle.com/lava18/google-play-store-apps/discussion), and we can see that one of the [discussions](https://www.kaggle.com/lava18/google-play-store-apps/discussion/66015) describes an error for a certain row.

In [18]:
print(android[0])
print(android[10472])
print(android[10473])

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
['Xposed Wi-Fi-Pwd', 'PERSONALIZATION', '3.5', '1042', '404k', '100,000+', 'Free', '0', 'Everyone', 'Personalization', 'August 5, 2014', '3.0.0', '4.0.3 and up']
['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


*As can be seen above for row 10473, the column category is missing and each column entry gets left shifted. Below we will delete that row*

In [19]:
del android[10473]


Next we find duplicate apps in the android dataset and sample a few


In [20]:
duplicate_apps=[]
unique_apps=[]

for app in android:
    name=app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)
print('Number of duplicate apps:',len(duplicate_apps))
print('\n')
print('Example of duplicate apps:', duplicate_apps[:15])

Number of duplicate apps: 1181


Example of duplicate apps: ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack', 'FreshBooks Classic', 'Insightly CRM', 'QuickBooks Accounting: Invoicing & Expenses', 'HipChat - Chat Built for Teams', 'Xero Accounting Software']



There can be many ways to remove these duplicates. One way is to randomly remove all duplicates. However, we will remove keep the value with highest number of reviews and delete rest of the duplicates.

In [21]:
reviews_max={}

for app in android[1:]:
    name=app[0]
    n_reviews=float(app[3])
    if name in reviews_max and reviews_max[name]<n_reviews:
        reviews_max[name]=n_reviews
    if name not in reviews_max:
        reviews_max[name]=n_reviews

In [27]:
len(reviews_max)

9659

In [28]:
android_clean=[]
already_added=[]

for app in android[1:]:
    name=app[0]
    n_reviews=float(app[3])
    if n_reviews==reviews_max[name] and name not in already_added:
        android_clean.append(app)
        already_added.append(name)
    

In [29]:
len(already_added)

9659

In [30]:
len(android_clean)

9659

*Next we will remove Non-English apps*

In [31]:
def is_english(string):
    for character in string:
        if ord(character)>127:
            return False
    return True


In [32]:
is_english('Instagram')

True

In [33]:
is_english('爱奇艺PPS -《欢乐颂2》电视剧热播') 

False

In [34]:
is_english('Docs To Go™ Free Office Suite')

False

In [35]:
is_english('Instachat 😜')

False

In [36]:
def is_english(string):
    char_count=0
    for character in string:
        if ord(character)>127:
            char_count+=1
    if char_count > 3:
        return False
    return True


In [37]:
is_english('爱奇艺PPS -《欢乐颂2》电视剧热播') 

False

In [39]:
is_english('Docs To Go™ Free Office Suite')

True

In [40]:
is_english('Instachat 😜')

True

In [60]:
android_english=[]
iphone_english=[]
for app in android_clean:
    if is_english(app[0]):
        android_english.append(app)
for app in iphone:
    if is_english(app[1]):
        iphone_english.append(app)

In [61]:
print(len(android_english))
print(len(iphone_english))

9614
6184


In [62]:
android_final=[]
iphone_final=[]
for app in android_english:
    if app[7]=='0':
        android_final.append(app)
for app in iphone_english:
    if app[4]=='0.0':
        iphone_final.append(app)


In [63]:
print(len(android_final))
print(len(iphone_final))

8864
3222


In [66]:
print(android_header)
print('\n')
print(iphone_header)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


## Creating Frequency Tables

In [67]:
def freq_table(dataset,index):
    table={}
    total=0
    for row in dataset:
        total+=1
        if row[index] in table:
            table[row[index]]+=1
        else:
            table[row[index]]=1
    table_percentages={}
    for key in table:
        table_percentages[key]=table[key]/total*100
    return table_percentages

In [68]:
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

In [70]:
display_table(android_final,1)

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

In [71]:
display_table(android_final,9)

Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.531137184115524
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2378158844765346
Action : 3.1024368231046933
Health & Fitness : 3.0798736462093865
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.8501805054151623
Video Players & Editors : 1.7712093862815883
Casual : 1.7599277978339352
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & Demo : 0.9363718411552346
Auto & Vehicles : 0.9250902527075

In [72]:
display_table(iphone_final,11)

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


In [78]:
unique_genres=freq_table(iphone_final,11)
for genre in unique_genres:
    total=0 #sum of number of ratings
    len_genre=0 #number of apps specific to each genre
    
    for app in iphone_final:
        genre_app=app[11]
        if genre_app==genre:
            ratings=float(app[5])
            total+=ratings
            len_genre+=1
    
    avg_rating=total/len_genre
    print(genre,':',avg_rating)
    

Business : 7491.117647058823
Games : 22788.6696905016
Reference : 74942.11111111111
Shopping : 26919.690476190477
Navigation : 86090.33333333333
Lifestyle : 16485.764705882353
Travel : 28243.8
Weather : 52279.892857142855
Music : 57326.530303030304
News : 21248.023255813954
Utilities : 18684.456790123455
Health & Fitness : 23298.015384615384
Book : 39758.5
Entertainment : 14029.830708661417
Productivity : 21028.410714285714
Finance : 31467.944444444445
Social Networking : 71548.34905660378
Food & Drink : 33333.92307692308
Catalogs : 4004.0
Photo & Video : 28441.54375
Medical : 612.0
Education : 7003.983050847458
Sports : 23008.898550724636
